In [20]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
   
# tranforming json file into a pandas dataframe library
import json # library to handle JSON files
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [21]:
import csv
import requests

In [22]:
CLIENT_ID = 'JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN' # your Foursquare ID
CLIENT_SECRET = 'U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN
CLIENT_SECRET:U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF


# London

In [23]:
"""Create CSVs from all tables on a Wikipedia article."""

import csv
import os
import platform

from bs4 import BeautifulSoup
import requests

def scrape(url, output_name):
    """Create CSVs from all tables in a Wikipedia article.
    ARGS:
        url (str): The full URL of the Wikipedia article to scrape tables from.
        output_name (str): The base file name (without filepath) to write to.
    """

    # Read tables from Wikipedia article into list of HTML strings
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    table_classes = {"class": ["sortable", "plainrowheaders"]}
    wikitables = soup.findAll("table", table_classes)

    # Create folder for output if it doesn't exist
    try:
        os.mkdir(output_name)
    except Exception:  # Generic OS Error
        pass

    for index, table in enumerate(wikitables):
        # Make a unique file name for each CSV
        if index == 0:
            filename = output_name
        else:
            filename = output_name + '_' + str(index)

        filepath = os.path.join(output_name, filename) + '.csv'

        with open(filepath, mode='w', newline='', encoding='utf-8') as output:
            # Deal with Windows inserting an extra '\r' in line terminators
            if platform.system() == 'Windows':
                kwargs = {'lineterminator': '\n'}

                csv_writer = csv.writer(output,
                                        quoting=csv.QUOTE_ALL,
                                        **kwargs)
            else:
                csv_writer = csv.writer(output,
                                        quoting=csv.QUOTE_ALL)

            write_html_table_to_csv(table, csv_writer)


def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.
    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """

    # Hold elements that span multiple rows in a list of
    # dictionaries that track 'rows_left' and 'value'
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])

        # If the first row, use it to define width of table
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        # Insert values from cells that span into this row
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    # Insert the data from previous row; decrement rows left
                    value = rowspan_data['value']
                    cells.insert(index, value)

                    if saved_rowspans[index]['rows_left'] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]['rows_left'] -= 1

        # If an element with rowspan, save it for future cells
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {
                    'rows_left': int(cell["rowspan"]),
                    'value': cell,
                }
                saved_rowspans[index] = rowspan_data

        if cells:
            # Clean the data of references and unusual whitespace
            cleaned = clean_data(cells)

            # Fill the row with empty columns if some are missing
            # (Some HTML tables leave final empty cells without a <td> tag)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing

            writer.writerow(cleaned)


def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.
    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.
    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """

    cleaned_cells = []

    for cell in row:
        # Strip references from the cell
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()

        # Strip sortkeys from the cell
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()

        # Strip footnotes from text and join into a single string
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != '[']

        cleaned = (
            ''.join(no_footnotes)  # Combine elements into single string
            .replace('\xa0', ' ')  # Replace non-breaking spaces
            .replace('\n', ' ')  # Replace newlines
            .strip()
        )

        cleaned_cells += [cleaned]

    return cleaned_cells

In [24]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
csv_file = 'London'
scrape(WIKI_URL, csv_file)
print('CSV Created')

CSV Created


In [37]:

fname1 = 'London'
fil2 = 'London_Indian_Population'
fl2 = os.path.join(fname1, fil2) + '.csv'
print(fl2)
ind_population = pd.read_csv(fl2, sep=',')
ind_population.head()

London/London_Indian_Population.csv


,Community,Indian population %
0,Harrow,26.4
1,Hounslow,19.0
2,Brent,18.6
3,Redbridge,16.4
4,Ealing,14.3


In [39]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
wikipedia_page = requests.get(wikipedia_link)

soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
popLn = pd.DataFrame(columns = columns)

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        popLn = popLn.append(pd.Series(values, index = columns), ignore_index = True)
        popLn

popLn.head(1)

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25


In [40]:
#Data Clean Up
#popLn = popLn.drop('Postcode', axis=1).join(popLn['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))
popLn = popLn[['Borough','Area (sq mi)','Population (2013 est)[1]']].reset_index(drop=True)
popLn['Borough'] = popLn['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
popLn.shape
#London = popLn

popLn.head()
popLn

,Borough,Area (sq mi),Population (2013 est)[1]
0,Barking and Dagenham [note,13.93,"194,352"
1,Barnet,33.49,"369,088"
2,Bexley,23.38,"236,687"
3,Brent,16.70,"317,264"
4,Bromley,57.97,"317,899"
5,Camden,8.40,"229,719"
6,Croydon,33.41,"372,752"
7,Ealing,21.44,"342,494"
8,Enfield,31.74,"320,524"
9,Greenwich [note,18.28,"264,008"


In [41]:
popLn.columns = ['Borough','Area','Population']
popLn['Population'] = popLn['Population'].map(lambda x: x.rstrip(',').rstrip(',').rstrip(','))
popLn

popLn["Population"] = popLn["Population"].str.replace(",","").astype(float)
popLn["Population"] = pd.to_numeric(popLn["Population"])
popLn["Area"] = pd.to_numeric(popLn["Area"])


In [42]:

print('Borough with Maximum Population : ' )
print(popLn.loc[popLn['Population'].idxmax()])
print('' )
print('Borough with Maximum Area : ' )
print(popLn.loc[popLn['Area'].idxmax()])



Borough with Maximum Population : 
Borough       Croydon
Area            33.41
Population     372752
Name: 6, dtype: object

Borough with Maximum Area : 
Borough       Bromley
Area            57.97
Population     317899
Name: 4, dtype: object


In [43]:
print('Borough with Maximum Indian Population : ' )
print(ind_population.loc[ind_population['Indian population %'].idxmax()])

#ind_population[['Indian population %']] = ind_population[['Indian population %']].apply(pd.to_numeric) 
#ind_population.sort_values(by=['Indian population %'], ascending=False)
ind_population.head()

Borough with Maximum Indian Population : 
Community              Harrow
Indian population %      26.4
Name: 0, dtype: object


,Community,Indian population %
0,Harrow,26.4
1,Hounslow,19.0
2,Brent,18.6
3,Redbridge,16.4
4,Ealing,14.3


#Based on the Maximum Population & Area, we have selected Croydon & Bromley Borough to Analyze

In [48]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
wikipedia_page = requests.get(wikipedia_link)

soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
df = pd.DataFrame(columns = columns)

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)
        df

df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [1]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[2]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[2],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[2],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [49]:
#Data Clean Up
df.columns = ['Location','Borough','Post-town','Postcode','Dial-code','Ref']
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))
df = df[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)
df.shape
#df.head()

(638, 4)

In [51]:
df.head()

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Addington,Croydon,CR0,CROYDON
4,Addiscombe,Croydon,CR0,CROYDON


In [52]:
#Ln = df[df['Post-town'].str.contains('LONDON')]
#Ln = df[df['Borough'].str.contains('Croydon' )]
#Ln = df[df['Borough'].isin(['Croydon', 'Bromley']) ]
Ln = df[df['Borough'].isin(['Harrow']) ]
Ln = Ln[['Location','Borough','Postcode','Post-town']].reset_index(drop=True)
Ln.Postcode = Ln.Postcode.str.strip()
print(Ln.shape)
Ln.head()

(14, 4)


,Location,Borough,Postcode,Post-town
0,Belmont,Harrow,HA3,"HARROW, STANMORE"
1,Belmont,Harrow,HA7,"HARROW, STANMORE"
2,Harrow,Harrow,HA1,HARROW
3,Harrow on the Hill,Harrow,HA1,HARROW
4,Harrow Weald,Harrow,HA3,HARROW


In [53]:
crLn = Ln.reset_index(drop=True)
crLn.columns = ['Neighborhood','Borough','Postcode','Post-town']
print(crLn.shape)
crLn.loc[crLn['Neighborhood'].isin(['Widmore (also Widmore Green)'])]='Widmore'
crLn
#crLn.head()


(14, 4)


,Neighborhood,Borough,Postcode,Post-town
0,Belmont,Harrow,HA3,"HARROW, STANMORE"
1,Belmont,Harrow,HA7,"HARROW, STANMORE"
2,Harrow,Harrow,HA1,HARROW
3,Harrow on the Hill,Harrow,HA1,HARROW
4,Harrow Weald,Harrow,HA3,HARROW
5,Hatch End,Harrow,HA5,PINNER
6,North Harrow,Harrow,HA1,HARROW
7,North Harrow,Harrow,HA2,HARROW
8,Pinner,Harrow,HA5,PINNER
9,Rayners Lane,Harrow,HA5,PINNER


In [54]:

def get_latlon(loc_ln):
    geolocator = Nominatim(user_agent="Ln_explorer")
    location1 = geolocator.geocode(loc_ln)
    latitude = location1.latitude
    longitude = location1.longitude
    co_ordin = [latitude, longitude]
    #print(loc_ln)
    return co_ordin

In [56]:
addington = get_latlon('addington')
WestHarrow = get_latlon('West Harrow')
print(addington)
print(WestHarrow)

[51.9509273, -0.9212331]
[51.5795852, -0.3530692]


In [57]:
location = crLn['Neighborhood']
coordinates = [get_latlon(location) for location in location.tolist()]

In [61]:
crLnd = crLn
# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
crLnd_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
crLnd['Latitude'] = crLnd_coordinates['Latitude']
crLnd['Longitude'] = crLnd_coordinates['Longitude']

crLnd

,Neighborhood,Borough,Postcode,Post-town,Latitude,Longitude
0,Belmont,Harrow,HA3,"HARROW, STANMORE",45.877912,5.652908
1,Belmont,Harrow,HA7,"HARROW, STANMORE",45.877912,5.652908
2,Harrow,Harrow,HA1,HARROW,51.596769,-0.337275
3,Harrow on the Hill,Harrow,HA1,HARROW,51.579270,-0.336656
4,Harrow Weald,Harrow,HA3,HARROW,51.604786,-0.340485
5,Hatch End,Harrow,HA5,PINNER,51.608440,-0.373548
6,North Harrow,Harrow,HA1,HARROW,51.585162,-0.363176
7,North Harrow,Harrow,HA2,HARROW,51.585162,-0.363176
8,Pinner,Harrow,HA5,PINNER,51.596871,-0.377014
9,Rayners Lane,Harrow,HA5,PINNER,51.576714,-0.370300


In [59]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(crLnd['Borough'].unique()),
        crLnd.shape[0]
    )
)

The dataframe has 1 boroughs and 14 neighborhoods.


In [62]:
lnBeck = crLnd
lnBeck.loc[13, 'Neighborhood']
neighborhood_latitude = lnBeck.loc[13, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = lnBeck.loc[13, 'Longitude'] # neighborhood longitude value

neighborhood_name = lnBeck.loc[13, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                              neighborhood_longitude))

Latitude and longitude values of West Harrow are 51.5795852, -0.3530692.


In [63]:
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cba8b07db04f530c423acd4'},
 'response': {'headerLocation': 'West Harrow',
  'headerFullLocation': 'West Harrow, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 51.58858520900001,
    'lng': -0.33861343075741873},
   'sw': {'lat': 51.570585190999985, 'lng': -0.3675249692425813}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd1ab0b046076b074307271',
       'name': 'West Harrow Park',
       'location': {'address': 'The Ridgeway',
        'lat': 51.577553917349476,
        'lng': -0.354538592936466,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.577553917349476,
          'lng': -0.354538592936466}],
        'distance': 247,
        'postalCode':

In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [65]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,West Harrow Park,Park,51.577554,-0.354539
1,Harrow Recreation Ground,Park,51.585777,-0.345683
2,Hundred Degrees UK Ltd,Indian Restaurant,51.581070,-0.348129
3,Twist Ice Cream,Ice Cream Shop,51.586708,-0.361651
4,Kingsfield Arms,Pub,51.575249,-0.343273


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [40]:
nearby_venues_unique = nearby_venues['categories'].value_counts().to_frame(name='Count')

nearby_venues_unique.head(5)

,Count
Coffee Shop,4
Park,3
Pub,3
Italian Restaurant,2
Mediterranean Restaurant,2


In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.head()
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
london_venues = getNearbyVenues(names=crLnd['Neighborhood'],
                                   latitudes=crLnd['Latitude'],
                                   longitudes=crLnd['Longitude']
                                  )

Belmont
Belmont
Harrow
Harrow on the Hill
Harrow Weald
Hatch End
North Harrow
North Harrow
Pinner
Rayners Lane
South Harrow
Stanmore
Wealdstone
West Harrow


In [68]:

print(london_venues.shape)
london_venues.head()

(148, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harrow,51.596769,-0.337275,Costa Coffee,51.596665,-0.335510,Coffee Shop
1,Harrow,51.596769,-0.337275,Everest Lounge,51.594233,-0.332192,Indian Restaurant
2,Harrow,51.596769,-0.337275,Subway,51.594388,-0.335041,Sandwich Place
3,Harrow,51.596769,-0.337275,Sainsbury's Local,51.593425,-0.334908,Convenience Store
4,Harrow,51.596769,-0.337275,Umainah International,51.599482,-0.342626,Clothing Store


In [69]:
london_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Harrow,5,5,5,5,5,5
Harrow Weald,6,6,6,6,6,6
Harrow on the Hill,30,30,30,30,30,30
Hatch End,12,12,12,12,12,12
North Harrow,16,16,16,16,16,16
Pinner,16,16,16,16,16,16
Rayners Lane,20,20,20,20,20,20
South Harrow,17,17,17,17,17,17
Stanmore,12,12,12,12,12,12


In [70]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 47 uniques categories.


In [71]:
london_venues_unique_count = london_venues['Venue Category'].value_counts().to_frame(name='Count')
london_venues_unique_count.head(10)

,Count
Coffee Shop,17
Indian Restaurant,13
Grocery Store,9
Pizza Place,8
Sandwich Place,7
Park,6
Supermarket,6
Italian Restaurant,6
Fast Food Restaurant,6
Gym / Fitness Center,6


In [72]:
address = 'Harrow on the Hill'

geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5792702, -0.3366561.


In [98]:
map_london = folium.Map(location = [latitude, longitude], zoom_start = 12)
map_london

In [74]:
lnBeck_df = lnBeck.loc[lnBeck['Neighborhood'] == 'Harrow on the Hill'] 

In [99]:
# Adding markers to map
for lat, lng, borough, loc in zip(crLnd['Latitude'], 
                                  crLnd['Longitude'],
                                  crLnd['Borough'],
                                  crLnd['Neighborhood']):
    label = '{} - {}'.format(loc, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_london)  
    
display(map_london)

In [76]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]
print(london_onehot.shape)
london_onehot.head()


(148, 48)


,Neighborhood,Bakery,Bar,Beer Store,Betting Shop,Bookstore,Burger Joint,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Donut Shop,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gas Station,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Lawyer,Metro Station,Middle Eastern Restaurant,Movie Theater,North Indian Restaurant,Park,Pizza Place,Platform,Portuguese Restaurant,Pub,Sandwich Place,Seafood Restaurant,Sri Lankan Restaurant,Supermarket,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Harrow,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harrow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harrow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Harrow,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harrow,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped.shape

(11, 48)

In [78]:
london_grouped.loc[london_grouped['Indian Restaurant'] != 0]

,Neighborhood,Bakery,Bar,Beer Store,Betting Shop,Bookstore,Burger Joint,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Donut Shop,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Furniture / Home Store,Gas Station,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Lawyer,Metro Station,Middle Eastern Restaurant,Movie Theater,North Indian Restaurant,Park,Pizza Place,Platform,Portuguese Restaurant,Pub,Sandwich Place,Seafood Restaurant,Sri Lankan Restaurant,Supermarket,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Harrow,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.2,0.200000,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0
1,Harrow Weald,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.166667,0.0,0.00,0.0,0.0
6,Rayners Lane,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.05,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.150000,0.000000,0.000000,0.0,0.300000,0.0,0.000000,0.05,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,0.050000,0.050000,0.0,0.000000,0.000000,0.000000,0.0,0.05,0.0,0.0
7,South Harrow,0.058824,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.058824,0.000000,0.0,0.0,0.0,0.117647,0.0,0.0,0.117647,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.058824,0.0,0.000000,0.00,0.058824,0.000000,0.0,0.058824,0.058824,0.058824,0.000000,0.058824,0.058824,0.058824,0.0,0.058824,0.117647,0.000000,0.0,0.00,0.0,0.0
8,Stanmore,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.250000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.0,0.083333,0.0,0.083333,0.00,0.000000,0.083333,0.0,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.083333,0.0,0.000000,0.083333,0.000000,0.0,0.00,0.0,0.0
9,Wealdstone,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.222222,0.222222,0.0,0.111111,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0
10,West Harrow,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.40,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.0,0.200000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0


In [79]:
num_top_venues = 10

for hood in london_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harrow,Coffee Shop,Convenience Store,Indian Restaurant,Sandwich Place,Clothing Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant
1,Harrow Weald,Indian Restaurant,Park,Thai Restaurant,Supermarket,Grocery Store,Beer Store,Deli / Bodega,Gas Station,Furniture / Home Store,French Restaurant
2,Harrow on the Hill,Coffee Shop,Clothing Store,Portuguese Restaurant,Gym,Pizza Place,Women's Store,Middle Eastern Restaurant,Bar,Bookstore,Department Store
3,Hatch End,Deli / Bodega,Chinese Restaurant,Burger Joint,Grocery Store,Furniture / Home Store,Italian Restaurant,Pizza Place,Pub,Greek Restaurant,Seafood Restaurant
4,North Harrow,Gym / Fitness Center,Park,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Grocery Store,Gas Station,Furniture / Home Store,French Restaurant


In [82]:
london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

In [83]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 2, 2, 0, 2, 4, 2, 2, 0], dtype=int32)

In [84]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = crLnd

# match/merge SE London data with latitude/longitude for each neighborhood
london_merged_latlong = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
#cols = ['Cluster Labels']
#london_merged_latlong[cols] = (london_merged_latlong[cols]//1)
london_merged_latlong.head()

,Neighborhood,Borough,Postcode,Post-town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belmont,Harrow,HA3,"HARROW, STANMORE",45.877912,5.652908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Belmont,Harrow,HA7,"HARROW, STANMORE",45.877912,5.652908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Harrow,Harrow,HA1,HARROW,51.596769,-0.337275,1.0,Coffee Shop,Convenience Store,Indian Restaurant,Sandwich Place,Clothing Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant
3,Harrow on the Hill,Harrow,HA1,HARROW,51.579270,-0.336656,2.0,Coffee Shop,Clothing Store,Portuguese Restaurant,Gym,Pizza Place,Women's Store,Middle Eastern Restaurant,Bar,Bookstore,Department Store
4,Harrow Weald,Harrow,HA3,HARROW,51.604786,-0.340485,4.0,Indian Restaurant,Park,Thai Restaurant,Supermarket,Grocery Store,Beer Store,Deli / Bodega,Gas Station,Furniture / Home Store,French Restaurant


In [89]:
london_merged_latlong=london_merged_latlong.dropna()
london_merged_latlong
#fname3 = 'London'
#fil3 = 'london_merged_latlong'
#fl3 = os.path.join(fname3, fil3) + '.csv'
#print(fl3)
#london_merged_latlong.to_csv(fl3, sep=',', encoding='utf-8', index=False)

London/london_merged_latlong.csv


In [129]:

fname3 = 'London'
fil3 = 'london_merged_latlong_edited'
fl3 = os.path.join(fname3, fil3) + '.csv'
print(fl3)
london_merged_latlong = pd.read_csv(fl3, sep=',')
london_merged_latlong

London/london_merged_latlong_edited.csv


,Neighborhood,Borough,Postcode,Post-town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harrow,Harrow,HA1,HARROW,51.596769,-0.337275,1,Coffee Shop,Convenience Store,Indian Restaurant,Sandwich Place,Clothing Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant
1,Harrow on the Hill,Harrow,HA1,HARROW,51.579270,-0.336656,2,Coffee Shop,Clothing Store,Portuguese Restaurant,Gym,Pizza Place,Women's Store,Middle Eastern Restaurant,Bar,Bookstore,Department Store
2,Harrow Weald,Harrow,HA3,HARROW,51.604786,-0.340485,4,Indian Restaurant,Park,Thai Restaurant,Supermarket,Grocery Store,Beer Store,Deli / Bodega,Gas Station,Furniture / Home Store,French Restaurant
3,Hatch End,Harrow,HA5,PINNER,51.608440,-0.373548,2,Deli / Bodega,Chinese Restaurant,Burger Joint,Grocery Store,Furniture / Home Store,Italian Restaurant,Pizza Place,Pub,Greek Restaurant,Seafood Restaurant
4,North Harrow,Harrow,HA1,HARROW,51.585162,-0.363176,0,Gym / Fitness Center,Park,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Grocery Store,Gas Station,Furniture / Home Store,French Restaurant
5,North Harrow,Harrow,HA2,HARROW,51.585162,-0.363176,0,Gym / Fitness Center,Park,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Grocery Store,Gas Station,Furniture / Home Store,French Restaurant
6,Pinner,Harrow,HA5,PINNER,51.596871,-0.377014,2,Italian Restaurant,Coffee Shop,Wine Bar,Pizza Place,Café,Bus Stop,Sandwich Place,Pub,Bookstore,Supermarket
7,Rayners Lane,Harrow,HA5,PINNER,51.576714,-0.370300,4,Indian Restaurant,Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Pub,Café,Lawyer,Fast Food Restaurant,Betting Shop
8,South Harrow,Harrow,HA2,HARROW,51.564652,-0.352221,2,Furniture / Home Store,Fast Food Restaurant,Supermarket,Portuguese Restaurant,Indian Restaurant,Coffee Shop,Metro Station,Park,Pizza Place,Bakery
9,Stanmore,Harrow,HA7,STANMORE,51.617421,-0.309511,2,Coffee Shop,Gas Station,Italian Restaurant,Pizza Place,Platform,Portuguese Restaurant,Indian Restaurant,Sandwich Place,Supermarket,Middle Eastern Restaurant


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harrow,-0.340485,4,Indian Restaurant,Park,Thai Restaurant,Supermarket,Grocery Store,Beer Store,Deli / Bodega,Gas Station,Furniture / Home Store,French Restaurant
7,Harrow,-0.370300,4,Indian Restaurant,Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Pub,Café,Lawyer,Fast Food Restaurant,Betting Shop


In [117]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged_latlong['Latitude'], london_merged_latlong['Longitude'], london_merged_latlong['Neighborhood'], london_merged_latlong['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

display(map_clusters)

In [118]:
# Cluster 1
london_merged_latlong.loc[london_merged_latlong['Cluster Labels'] == 0, london_merged_latlong.columns[[1] + list(range(5, london_merged_latlong.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Harrow,-0.363176,0,Gym / Fitness Center,Park,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Grocery Store,Gas Station,Furniture / Home Store,French Restaurant
5,Harrow,-0.363176,0,Gym / Fitness Center,Park,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Coffee Shop,Grocery Store,Gas Station,Furniture / Home Store,French Restaurant
10,Harrow,-0.329476,0,Gym / Fitness Center,Gym,Sandwich Place,Indian Restaurant,Convenience Store,Platform,Park,Bus Stop,Department Store,Bar


In [123]:
# Cluster 2
london_merged_latlong.loc[london_merged_latlong['Cluster Labels'] == 1, london_merged_latlong.columns[[1] + list(range(5, london_merged_latlong.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harrow,-0.337275,1,Coffee Shop,Convenience Store,Indian Restaurant,Sandwich Place,Clothing Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant


In [124]:
# Cluster 3
london_merged_latlong.loc[london_merged_latlong['Cluster Labels'] == 2, london_merged_latlong.columns[[1] + list(range(5, london_merged_latlong.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Harrow,-0.336656,2,Coffee Shop,Clothing Store,Portuguese Restaurant,Gym,Pizza Place,Women's Store,Middle Eastern Restaurant,Bar,Bookstore,Department Store
3,Harrow,-0.373548,2,Deli / Bodega,Chinese Restaurant,Burger Joint,Grocery Store,Furniture / Home Store,Italian Restaurant,Pizza Place,Pub,Greek Restaurant,Seafood Restaurant
6,Harrow,-0.377014,2,Italian Restaurant,Coffee Shop,Wine Bar,Pizza Place,Café,Bus Stop,Sandwich Place,Pub,Bookstore,Supermarket
8,Harrow,-0.352221,2,Furniture / Home Store,Fast Food Restaurant,Supermarket,Portuguese Restaurant,Indian Restaurant,Coffee Shop,Metro Station,Park,Pizza Place,Bakery
9,Harrow,-0.309511,2,Coffee Shop,Gas Station,Italian Restaurant,Pizza Place,Platform,Portuguese Restaurant,Indian Restaurant,Sandwich Place,Supermarket,Middle Eastern Restaurant


In [125]:
# Cluster 4
london_merged_latlong.loc[london_merged_latlong['Cluster Labels'] == 3, london_merged_latlong.columns[[1] + list(range(5, london_merged_latlong.shape[1]))]]


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Harrow,-0.353069,3,Café,Park,Indian Restaurant,Grocery Store,Women's Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant


In [128]:
# Cluster 5
london_merged_latlong.loc[london_merged_latlong['Cluster Labels'] == 4, london_merged_latlong.columns[[1] + list(range(5, london_merged_latlong.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harrow,-0.340485,4,Indian Restaurant,Park,Thai Restaurant,Supermarket,Grocery Store,Beer Store,Deli / Bodega,Gas Station,Furniture / Home Store,French Restaurant
7,Harrow,-0.370300,4,Indian Restaurant,Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Pub,Café,Lawyer,Fast Food Restaurant,Betting Shop


## Conlcusion

The following are the highlights of Harrow Clusters:

1. Gyms/Fitness, Coffee Shops, Coffee Shops are popular in the Harrow.
2. As for Indian restaurants, We can the neighborhoods Rayners Lane  and Harrow Weald  which top in the Indians visited venues.
3. Also to note, West Harrow & South Harrow are having second top Indians visted venues which are close to Rayners Lanes. But Harrow Weald is comparatively far from said 3 neighborhoods.

Considering all the analysis with the available Data - Below are the neighborhoods in the priority order to open Indian Restaurants:
1. Rayners Lane
2. West Harrow
3. Harrow Weald


Below are the few other factors in terms of data would have given better results with more insight into the best location. 
1. Real Estate 
2. Crime data
3. Income per capita
4. Traffic Data
5. More venues exploration with the Foursquare
6. Ratings and feedback of the current restaurants within the clusters 

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harrow,-0.337275,1,Coffee Shop,Convenience Store,Indian Restaurant,Sandwich Place,Clothing Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Harrow,-0.336656,2,Coffee Shop,Clothing Store,Portuguese Restaurant,Gym,Pizza Place,Women's Store,Middle Eastern Restaurant,Bar,Bookstore,Department Store
3,Harrow,-0.373548,2,Deli / Bodega,Chinese Restaurant,Burger Joint,Grocery Store,Furniture / Home Store,Italian Restaurant,Pizza Place,Pub,Greek Restaurant,Seafood Restaurant
6,Harrow,-0.377014,2,Italian Restaurant,Coffee Shop,Wine Bar,Pizza Place,Café,Bus Stop,Sandwich Place,Pub,Bookstore,Supermarket
8,Harrow,-0.352221,2,Furniture / Home Store,Fast Food Restaurant,Supermarket,Portuguese Restaurant,Indian Restaurant,Coffee Shop,Metro Station,Park,Pizza Place,Bakery
9,Harrow,-0.309511,2,Coffee Shop,Gas Station,Italian Restaurant,Pizza Place,Platform,Portuguese Restaurant,Indian Restaurant,Sandwich Place,Supermarket,Middle Eastern Restaurant


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Harrow,-0.353069,3,Café,Park,Indian Restaurant,Grocery Store,Women's Store,Gas Station,Furniture / Home Store,French Restaurant,Food & Drink Shop,Fast Food Restaurant
